# CS145: Project 2
## Part 2 | Exploring the GitHub Dataset with Colaboratory (60 points)
---

### Notes (read carefully!):

* Be sure you read the instructions on each cell and understand what it is doing before running it.
* Don't forget that if you can always re-download the starter notebook from the course website if you need to.
* You may create new cells to use for testing, debugging, exploring, etc., and this is in fact encouraged!
**Just make sure that the final answer for each question is _in its own cell_ and _clearly indicated_**.
* Colab will not warn you about how many bytes your SQL query will consume.  **Be sure to check on the BigQuery UI first before running queries here!**
* See the assignment handout for submission instructions.
* Have fun!

## Collaborators:
Please list the names and SUNet IDs of your collaborators below:
* *Name, SUNet ID*

## Setting Up BigQuery and Dependencies

Run the two cells below (shift + enter) to authenticate your project and import the libraries you'll need. 

Note that you need to fill in the `project_id` variable with the Google Cloud project id you are using for this course.  You can see your project ID by going to https://console.cloud.google.com/cloud-resource-manager.



In [0]:
# Run this cell to authenticate yourself to BigQuery.
from google.colab import auth
auth.authenticate_user()
project_id = "YOUR PROJECT ID HERE"

In [0]:
# Some imports you will need
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np

%matplotlib inline
plt.style.use('seaborn-whitegrid')

# Overview

As we saw in class, BigQuery has a massive dataset of GitHub files and statistics, including information about repositories, commits, and file contents.  In this part of the assignment, we will be working with this dataset further.  Don't worry if you are not too familiar with Git and GitHub -- we will explain everything you need to know to complete this part of the assignment.

---
## Notes

**The GitHub dataset available on BigQuery is actually quite massive. *A single query on the "contents" table alone (it is 2.16TB!) can eat up your 1TB allowance for the month AND cut into about 10% of your GCloud credit for the class.***

To make this part of the project more manageable, we have subset the original data.  We have preserved almost all information in the original tables, but we kept only the information on the top 500,000 most "watched" GitHub repos between January, 2016 and the most recent data as of October, 2018.

You can see the tables we will be working with [here](https://bigquery.cloud.google.com/dataset/cs145-ta:project_part2). **Read through the schemas to get familiar with the data**.  Note that some of the tables are still quite large (the contents table is about 500GB), so you should exercise the usual caution when working with them.  Before running queries on this notebook, it's good practice to first set up query limits on your BigQuery account or see how many bytes will be billed on the web UI.

---

## A Super Quick Primer on Git

If you are not very familiar with Git and GitHub, here are some high-level explanations that will give you enough context to get you through this part of the problem:

*  *GitHub*: GitHub is a source-control service provider.  GitHub allows you to collaborate on and keep track of source code in a fairly efficient way.

*  *commit*: A commit can be thought of as a change that is applied to some set of files. i.e., if some set of files is in some state A, you can make changes to A and *commit* your changes to the set of files so that it is now in state B.  A commit is identified by a *hash* of the information in your change (the author of the commit, who actually committed [i.e. applied] the changes to the set of files, the changes themselves, etc.)

* *parent commit*: The commit that came before your current commit.

*   *repo*:  A repo (short for repository) is GitHub's abstraction for a collection of files along with a history of commits on those files. If you have GitHub username "foo" and you make a repository called "data-rocks", your repo name will be "foo/data-rocks".  You can think of a repo's history in terms of its commits.  E.g., "foo/data-rocks" can have the set of "states" A->B->C->D, where each state change (A->B, B->C, C->D) was due to a commit.

*   *branch*:  To keep track of different commit histories, GitHub repos can have branches.  The 'main' branch (i.e. commit history) of the repo is called the 'master' branch.  Say on "foo/data-rocks" we have the commit history A->B->C->D on the master branch.  If someone else comes along and decides to add a cool new feature to "foo/data-rocks", they can create a branch called "cool-new-feature" that branches away from the master branch. All the code from the main branch will be there, but new code added to "cool-new-feature" will not be on the main branch.

*   *ref*: For the purpose of this assignment, you can think of the 'ref' field on the "files" table as referring to the branch in which a file lives in a repository at some point in time.



For the purposes of this question, you don't need to know about the following things in detail:


*   Commit trees
*   The encoding attribute on the commits table

If you want more clarifications about Git and GitHub in order to answer this question, be sure to post on Piazza or come to Office Hours.  In many cases, a quick web search will also help answer your question.

# Section 1 | Getting Familiar with the GitHub Data
---

## Understanding the GitHub Tables

As you saw in lecture, E/R diagrams are a way to convey the structure of a database (a set of tables) in a non-technical way that is still precise enough for us to work with technically.  As it turns out, E/R diagrams will prove to be a useful tool in getting familiar with the structure of our GitHub tables before we even analyze any of its data!

## Question 1: CS145 GitHub tables -->  E/R Diagram (6 points)

Using what you learned in lecture, create an E/R diagram for the data contained in the `cs145-ta:project_part2 github_repo` tables [here](https://bigquery.cloud.google.com/dataset/cs145-ta:project_part2).  For the purposes of this exercise, ignore the `github_repo_readme_contents_cs145` table (we'll be using that one later)

### Notes

* The CS145 GitHub Repo tables may not directly translate into an E/R diagram as described in class.  An important part of this question is for you to analyze the existing tables, think about the relationships between objects in the tables as a whole, and construct a reasonable E/R diagram from that analysis.

* Consider the "author" and "committer" attribute structures a single attribute each.  Importantly, you do not need to include `committer.name`, `committer.email`, etc.  in your E/R diagrams.  Note that the "language" attribute is actually an *array* type. Be sure to account for this in your E/R diagram.

* Your E/R diagram should be an image.  You can hand draw it or generate it with your favorite software.  You can include an image in your Colab Notebook as follows:



> 1. Add the image to your own Google Drive.  
> 2. Get a shareable link for your image, and note the "ID" field of the URL.
> > Your link will look something like "`https://drive.google.com/open?id=<some ID>`"
> 3. Add the following markup to your cell `![](https://drive.google.com/uc?export=view&id=<your image ID>)`
> 4. Run the cell


---
*TODO: insert your E/R diagram here*

---

## Question 2: Explain your E/R Diagram (4 points)

In a small paragraph, explain your E/R diagram.  You should cover at least:

*   what your entities are
*   what the relationships between those entities are (including whether they are one-to-many, many-to-one, etc.)
*   what your keys are for each entity

You should also include a quick explanation of how you determined the overall structure of your chart.

---
*TODO: insert explanation of your E/R diagram here*

---

## Question 3: Translate your E/R Diagram to a Set of Schemas (5 points)

Provide a set of schemas that translate directly to the E/R Diagram you generated in Question 2.  These schemas should not be the same as the schemas in the CS145 GitHub data tables. 

Be sure to specify in your schemas:
1. the ***name*** of each attribute (you don't need to worry about types)
2. the ***key for each schema***
3. ***Foreign keys in your schemas*** and what table they reference. 

---
*TODO: insert your proposed schemas here.*


*Example table for formatting reference:*


Column A | Column B | Column C
----------------|-----------------|-----------------
   value 1  |   value 2   |   value 3
   value 4  |   value 5   |   value 6

---

## Question 4: Analysis (4 points)

You now have two sets of schemas:  the ones you came up with in Question 3 based on your E/R diagram, and those that we provided for you.  

**In a single paragraph (no more than 100 words), comment on the tradeoffs between the two.  Which do you think is better?**

Use the discussions that we have had in class as a starting point for your analysis.   There is no single correct answer to this question.  Some people abide strongly to their E/R diagrams, while others decide to slice and dice what would have been on an E/R diagram to better fit their needs.


---
*TODO: insert your analysis of the two schemas here*

---

# Section 2 | Git Visualizing!
---

## Initial Exploration

Now that we understand the dataset we will be working with very well, let's get down to analyzing some of its properties. For the visualizations in this section, feel free to use whatever graphing library you are most comfortable with.  We recommend using one of the following:


*   seaborn (https://seaborn.pydata.org/tutorial.html)
*   matplotlib (https://matplotlib.org/3.0.0/tutorials/index.html)
*   Altair (https://altair-viz.github.io/)
*   Pandas (https://pandas.pydata.org/pandas-docs/stable/visualization.html)


> * **Note**: that you can plot directly from a Pandas DataFrame if you'd like (https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.plot.html)!





### How to BigQuery in Colab

Jupyter notebooks (what Colab notebooks are based on) have a concept called "magics".
If you write the following line at the top of a `Code` cell:

```
%%bigquery --project $project_id variable # this is the key line
SELECT ....
FROM ...
```

That "%%" converts the cell into a SQL cell. The resulting table that is generated is saved into `variable`. You can now use `variable` with your favorite visualization library to generate plots!

Run the following two cells to see this in action.

In [0]:
%%bigquery --project $project_id example

SELECT lrepo_name, watch_count
FROM `cs145-ta.project_part2.github_repos_cs145`
ORDER BY watch_count DESC
LIMIT 10;

In [0]:
example.head()

## Question 5: Count Breakdowns Across Various Fields (9 points)



Let's get out feet wet with this data by creating the following plots:

1.   License distribution across repos
2.   Language distribution across repos
3.   File size distribution across repos
4.   Distribution of files associated with a repo
5.   Number of commits by author and committer across repos
6.   The distribution of the length of commit messages across repos


Note that you will not receive full credit if your charts are  poorly made (i.e. very unclear or unreadable).


### Hints
* Some of these plots will need at least one of their axes to be log-scaled in order to be readable
* For more readable plots, you can use [pandas.DataFrame.sample](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.sample.html).  A sample size between 1,000 and 10,000 should give you more readable plots.

##  Reminders
* Be careful with your queries!  Don't run `SELECT *` blindly on a table in this Colab notebook since you will not get a warning of how much data the query will consume.  Always how much data a query will consume on the BigQuery UI first -- you are also better off setting a query limit as we described earlier. 
* Don't forget to use the subsetted GitHub tables we provide [here](https://bigquery.cloud.google.com/dataset/cs145-ta:project_part2), not the original ones on BigQuery. 




### a) License distribution (1 point)
#### (x-axis: license type, y-axis: # repos containing that license)


In [0]:
%%bigquery --project $project_id q5a

# YOUR QUERY HERE

In [0]:
# YOUR PLOT CODE HERE

### b) Language distribution (1 point)
#### (x-axis: programming language, y-axis: # repos containing at least one file in that language)

To keep the chart readable, only keep the top 20 languages.

**Hint:** https://cloud.google.com/bigquery/docs/reference/standard-sql/arrays


In [0]:
%%bigquery --project $project_id q5b

# YOUR QUERY HERE

In [0]:
# YOUR PLOT CODE HERE

### c) File size distribution (1 point)
#### (x-axis: file size, y-axis: # files of that size) 

In [0]:
%%bigquery --project $project_id q5c

# YOUR QUERY HERE

In [0]:
# YOUR PLOT CODE HERE

### d) Distribution of files associated with a repo (1 point)
#### (x-axis: # files associated with a repo, y-axis: # repos associated with that many files)

In [0]:
%%bigquery --project $project_id q5d

# YOUR QUERY HERE

In [0]:
# YOUR PLOT CODE HERE

### e) Number of commits by author and committer (1 point)
#### (x-axis: # commits, y-axis: # authors/committers with that many commits)

**Note**: for this chart, plot the curves for author and committers side-by-side for comparison.



In [0]:
%%bigquery --project $project_id q5e_authors

# YOUR QUERY HERE

In [0]:
%%bigquery --project $project_id q5e_committers

# YOUR QUERY HERE

In [0]:
# YOUR PLOT CODE HERE - AUTHORS

In [0]:
# YOUR PLOT CODE HERE - COMMITTERS

### f) The distribution of the length of commit messages (1 point)
#### (x-axis: length of the commit message, y-axis: # commits with that length)

**Note**: The query for this plot may use ~30GB of data.

In [0]:
%%bigquery --project $project_id q5f

# YOUR QUERY HERE

In [0]:
# YOUR PLOT CODE HERE

### g) In a single paragraph (no more than 100 words), describe and analyze the charts you just generated.  What are some interesting trends in the data?  Was there anything that you did not expect? (3 points)

---
*TODO: insert one paragraph analyzing the exploratory charts you generated*

---

## What Makes a Good Repo?

Given that we have some interesting data at our disposal, let's try to answer the question: what makes a good GitHub repo? For our purposes, a "good" repo is simply a repo with a high watch count; this refers to how many people are following the repo for updates.

To begin, let's see if any of the features *we've already explored* give us any good answers.

## Question 6: Using What We've Worked With (17 points)

Create plots for the following features in a repo and how they relate to that repo's watch count:

1.  License type
2.  Languages
3.  Average file size in a repo
4.  File count in a repo
5.  The number of "power" committers or authors a repo has
6. Average message length of commits on a repo


### a) License type (2 points)

In [0]:
%%bigquery --project $project_id q6a

# YOUR QUERY HERE

In [0]:
# YOUR PLOT CODE HERE

### b) Languages (2 points)



In [0]:
%%bigquery --project $project_id q6b

# YOUR QUERY HERE

In [0]:
# YOUR PLOT CODE HERE

### c) Average file size in a repo (2 points)
**Note:** For this question, you may use the `github_repo_readme_contents_cs145` table instead of the full contents table.


In [0]:
%%bigquery --project $project_id q6c

# YOUR QUERY HERE

In [0]:
# YOUR PLOT CODE HERE

### d) File count in a repo (2 points)

In [0]:
%%bigquery --project $project_id q6d

# YOUR QUERY HERE

In [0]:
# YOUR PLOT CODE HERE

### e) The number of "power" committers or authors a repo has (3 points)

 **Definition:** a "power" committer or author is an account that has committed/authored 1,000 or more commits.

In [0]:
%%bigquery --project $project_id q6e_power_committers

# YOUR QUERY HERE

In [0]:
# YOUR PLOT CODE HERE

In [0]:
%%bigquery --project $project_id q6e_power_authors

# YOUR QUERY HERE

In [0]:
# YOUR PLOT CODE HERE

### f) Average message length of commits on a repo. (3 points)

First make a plot of the average commit length of repositories against the number of repositories with that average commit length. Then, make a plot of how average commit length on a repository correlates to its watch count. Round to nearest byte.


In [0]:
%%bigquery --project $project_id q6f_avg_commit_length

# YOUR QUERY HERE

In [0]:
# YOUR PLOT CODE HERE

In [0]:
%%bigquery --project $project_id q6f_msg_length_watch_count

# YOUR QUERY HERE

In [0]:
# YOUR PLOT CODE HERE

### g) Which, if any, of the features we inspected above have the highest correlation with a repo having a high watch count? Does the answer make sense, or does it seem counterintuitive? Explain your answer in a small paragraph, no more than 200 words.  Be sure to cite the charts you generated. (3 points)

---
*TODO: insert your answer here*

---

## What Do Others Have to Say?

At this point we have learned a couple of things about how certain features may or may not impact the popularity of a GitHub repo.  However, we really only looked at features of GitHub repos that we had initially explored when we were getting a feel for the dataset!  There has got to be more things we can inspect than that.

If you do a web search for "how to make my git repo popular," you will find that more than a couple of people suggest investing time in your `README` file.  The `README` usually gives an overview to a GitHub project and may include other information about the codebase such as whether its most recent build passed or how to begin contributing to that repo.  [Here](https://github.com/vuejs/vue/blob/dev/README.md) is an example `README` file for the popular web development framework Vue.js.


## IMPORTANT: Note about Contents Table

Note that the original `github_repo_contents_cs145` table is about half a TB!  In order to save you the pain of using up 500GB of your credits to subset this table into a workable size for this problem, we have done it for you. 

## ***For the rest of this question, be sure that you use the `github_repo_readme_contents_cs145` table ***

## Question 7: Analyzing Features (15 points)

Analyze the following features of a repo's `README` file and how they relate to the popularity of a repository, generating an informative plot for each feature:

1.   Having or not having a `README` file
2.   The length of the `README` file
3.   The number of images (.jpg, .png, .svg) or GIFs (.gif) included in the `README`

Note that this part is intentionally more open ended than usual.  Part of what we are looking for is the thoroughness with which you leverage SQL and your plots to gain insight into these features. 

 **Hint:** BigQuery's SQL string, REGEX, and array functions will be helpful here.
 
 **Hint:** You can develop and test regexes on online testers like https://regex101.com/.

### a) Having or not having a `README` file (4 points)




In [0]:
%%bigquery --project $project_id q7a

# YOUR QUERY HERE

In [0]:
# YOUR PLOT CODE HERE

### b) The length of the `README` file (2 points)

You may ignore README files with length 0.  

**Note**: If a project has multiple `README` files, you can just take the average size of those files.

In [0]:
%%bigquery --project $project_id q7b

# YOUR QUERY HERE

In [0]:
# YOUR PLOT CODE HERE

### c) The number of images (`.jpg`, `.png`, `.svg`) or GIFs (`.gif`) included in the `README` (5 points)

#### Plot the total count of `.png`, `.jpg`, `.svg`, and `.gif` files included.  Remember you can sample the results on a DataFrame for clearer visualizations.

You can find whether an image/GIF is included in a `README` by searching for the following strings:
*   \!\[{*zero+ characters that are not* ']'}]({*zero+ chars that are not* ')'}.(*png|jpg|svg|gif*))
> Example: `![my_svg](https://foo.io/v/svgs/my_cool_svg.svg)`
* <{*zero+ whitespace*}*img*{*one+ whitespace*}src="{*zero+ chars not *'\"'}.(*png|jpg|svg|gif*)
> Example: `<img src="https://foo.bar/attachments/cat.gif" />`

**Hint**: You should test out your regular expressions on a subset of README files so that you can manually verify they work.

**Note**: If you end up dealing with multiple versions of the same `README.md` file, it's fine to average statistics among them.

In [0]:
%%bigquery --project $project_id q7c

# YOUR QUERY HERE

In [0]:
# YOUR PLOT CODE HERE


### d) Would you say that a "good" `README` is correlated with a popular repository, based on the features you studied?  Why or why not?  If you were to analyze more in-depth features on the `README` file for correlation with repo popularity what would they be? (4 points)

---
*TODO: insert one paragraph with your answer here*

---

## Optional Extra Credit

The problems below are entirely optional, though you may want to try them if you have time in preparation for the kinds of things you'll do in project 3.

## Extra Credit 1: What other features correlate with a highly watched repo? (6 points extra credit)

We studied only a handful of features that could correlate with a highly watched repo.  Can you find a few more that seem especially promising?  Back your proposed features with data and charts.


---

*List new features here*


---



In [0]:
# YOUR QUERIES HERE

In [0]:
# YOUR PLOT CODE HERE

## Extra Credit 2: Predicting Repo Watch Count (4 points extra credit)

Use linear regression to predict the watch count of a repo given a handful of its features.

BigQuery has built-in Machine Learning functionality that makes it straightforward to do basic Machine Learning tasks like linear and logistic regression.  Use a subset of the features that we've been working with on this problem to predict the watch count of a repo.

You will find the following resources helpful:

*   https://cloud.google.com/bigquery/docs/bigqueryml-intro
* https://cloud.google.com/bigquery/docs/bigqueryml-analyst-start
*   https://cloud.google.com/bigquery/docs/bigqueryml-ncaa

In your final output, be sure we can see the following values:

*   The repo name
*   The values of the features you are using
*   The predicted watch count
*   The actual watch count



In [0]:
# PREDICTION CODE HERE

## Closing Note

Understanding data through visualizations is crucial to any working data scientist or database engineer.  Yet, we only touched the tip of the iceberg with the visualizations we made here.  If you are interested in exploring the subject more, we recommend taking *CS 448B: Data Visualization*. 